In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import scvi
import gseapy

In [2]:
gseapy.get_library_name()

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'Azimuth_2023',
 'Azimuth_Cell_Types_2021',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'COVID-19_Related_Gene_Sets_2021',
 'Cancer_Cell_Line_Encyclopedia',
 'CellMarker_2024',
 'CellMarker_Augmented_2021',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'ChEA_2022',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'Descartes_Cell_Types_and_Tissue_2021',
 'Diabetes_Perturbatio

In [2]:
gene_sets_to_test = ['GO_Biological_Process_2023','GO_Cellular_Component_2023','GO_Molecular_Function_2023',
                    'WikiPathway_2023_Human'
                    'MSigDB_Hallmark_2020',
                    'KEGG_2021_Human']

## GSEA with up and down regulated DEGs for all cell types

In [7]:
file_path_diff_exp = '../../results/human_alcohol_brenner_et_al/scvi/'
file_path_gmt = '../../gene_sets/'
gsea_on = 'all_de_genes/'#'all_de_genes' or 'positive_regulated' or 'negative_regulated'
results_dir = '../../results/human_alcohol_brenner_et_al/GSEA/eneuro_review/'+gsea_on#Make sure correct DEG data frame is read as 'input_gsea'

for file in os.listdir(file_path_diff_exp):
    if 'tsv' in file and 'all' not in file and 'lfc' not in file:
        diff_exp = pd.read_csv(file_path_diff_exp+file, sep='\t',index_col=0)
        diff_exp.loc[:,'sign_lfc']=np.where(diff_exp['lfc_median']<0,-1,1)
        file = file.replace(" ","")
        cell_type = "".join(i for i in file.split('_')[3:]).replace('.tsv',"")
        print(cell_type)
        
        ######################## For all DEGs or separate +ve and -ve regulated genes############################
        col = np.where(diff_exp.columns=='log10_pscore')[0]
        diff_exp.loc[:,'metric_gsea'] = diff_exp['log10_pscore']*diff_exp['sign_lfc']
        input_gsea = pd.DataFrame(columns=['metric_gsea'])
        
        if gsea_on=='all_de_genes/':
            input_gsea = diff_exp.loc[:,'metric_gsea']
        #Positive DEGs:
        elif gsea_on=='positive_regulated/':
            input_gsea = diff_exp.iloc[list(np.where(diff_exp['lfc_median']>0)[0]),'metric_gsea']        
        #Negative DEGs
        elif gsea_on=='negative_regulated/':
            input_gsea = diff_exp.iloc[np.where(diff_exp.lfc_median<0)[0],'metric_gsea']
        
        input_gsea = input_gsea.sort_values(ascending=False)
        gsea_all_paths = pd.DataFrame()
        
        ## Exception handling for when the gmt gene set is irrelevant for the differentially expressed genes
        try:
            gsea_result = gseapy.prerank(rnk=input_gsea, gene_sets=gene_sets_to_test, organism='human')
        except LookupError:
            continue
        #filtered_sets = gsea_result.res2d.iloc[
                                            #np.where((gsea_result.res2d['NOM p-val']<=0.05)&
                                                    #(gsea_result.res2d['FDR q-val']<=0.25))[0]
                                            #,:]
        filtered_sets = gsea_result.res2d
        if len(filtered_sets)>0:
            gsea_all_paths = pd.concat([gsea_all_paths,filtered_sets]).reset_index(drop=True)
            gsea_all_paths=gsea_all_paths.drop_duplicates()
            gsea_all_paths.to_csv(results_dir+cell_type+'.tsv', sep='\t')

2024-06-23 00:50:22,016 [WARNING] Duplicated values found in preranked stats: 38.29% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


Chandelier


2024-06-23 00:50:32,642 [ERROR] No supported gene_sets: WikiPathway_2023_HumanMSigDB_Hallmark_2020
2024-06-23 00:50:44,767 [WARNING] Duplicated values found in preranked stats: 28.96% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


L5IT


2024-06-23 00:50:52,472 [ERROR] No supported gene_sets: WikiPathway_2023_HumanMSigDB_Hallmark_2020
2024-06-23 00:50:56,822 [WARNING] Duplicated values found in preranked stats: 32.19% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


...

2024-06-23 00:57:09,841 [ERROR] No supported gene_sets: WikiPathway_2023_HumanMSigDB_Hallmark_2020
2024-06-23 00:57:18,298 [WARNING] Duplicated values found in preranked stats: 64.49% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


SstChodl


2024-06-23 00:57:28,883 [ERROR] No supported gene_sets: WikiPathway_2023_HumanMSigDB_Hallmark_2020
2024-06-23 00:57:37,334 [WARNING] Duplicated values found in preranked stats: 34.03% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


Pax6


2024-06-23 00:57:47,920 [ERROR] No supported gene_sets: WikiPathway_2023_HumanMSigDB_Hallmark_2020
